# MVP Pipeline de Dados – Recursos Humanos  
## Análise de Retenção e Eficiência Organizacional  

**Vanessa Araújo**  

PUC-RJ – MBA em Ciência de Dados e Analytics  
Disciplina de Engenharia de Dados  

---

###Script para carga da camada BRONZE

###Imports
Imports das bibliotecas necessárias para o funcionamento do script.

In [0]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.types import StringType, IntegerType, DateType, FloatType
from pyspark.sql.functions import col, to_date, regexp_replace



###Carga de Dados
Os dados serão recuperados do GitHub, repositório público. O arquivo foi copiado do Kaggle para o GitHub.

Após carregados, as colunas são renomeadas, conforme o padrão da tabela. Em seguida, os dados são persistidos sem alterações, conforme padrão da camada bronze.

In [0]:
# Inicializa a sessão do Spark
spark = SparkSession.builder \
    .appName("Carregar CSV RHG para Delta") \
    .getOrCreate()

# Carga de Dados
# Recupera os dados do GitHub em formato CSV (UTF-8)
df_pandas = pd.read_csv(
    'https://raw.githubusercontent.com/VanessAraujo-3/mvp-rh-engenharia-dados/main/data/raw/HRDadosLimpos.csv',
    sep=',', encoding='utf-8', skip_blank_lines=True, on_bad_lines='skip'
)

# Remove espaços extras nos nomes das colunas
df_pandas.columns = df_pandas.columns.str.strip()

print("Dimensões do dataset:", df_pandas.shape)
df_pandas.head()

# Converte o DataFrame do Pandas em DataFrame do Spark
df_spark = spark.createDataFrame(df_pandas)

# Exibe o schema inicial do DataFrame do Spark
df_spark.printSchema()

# Renomeia colunas para seguir padrão snake_case
df_spark = df_spark \
    .withColumnRenamed("Nome do funcionário", "nome_funcionario") \
    .withColumnRenamed("IDFuncionario", "id_funcionario") \
    .withColumnRenamed("IDCasado", "id_casado") \
    .withColumnRenamed("IDStatusConjulgal", "id_status_conjulgal") \
    .withColumnRenamed("IDGenero", "id_genero") \
    .withColumnRenamed("IDStatusFuncionario", "id_status_funcionario") \
    .withColumnRenamed("IDepartamento", "id_departamento") \
    .withColumnRenamed("IDPontuacaoDesempenho", "id_pontuacao_desempenho") \
    .withColumnRenamed("IDRecrutamentoDiversidade", "id_recrutamento_diversidade") \
    .withColumnRenamed("Salario", "salario") \
    .withColumnRenamed("IDesligado", "id_desligado") \
    .withColumnRenamed("IDCargo", "id_cargo") \
    .withColumnRenamed("Cargo", "cargo") \
    .withColumnRenamed("Estado", "estado") \
    .withColumnRenamed("CEP", "cep") \
    .withColumnRenamed("ANIV", "aniversario") \
    .withColumnRenamed("Sexo", "sexo") \
    .withColumnRenamed("EstadoCivil", "estado_civil") \
    .withColumnRenamed("Cidadania", "cidadania") \
    .withColumnRenamed("Latino", "latino") \
    .withColumnRenamed("Raca", "raca") \
    .withColumnRenamed("Data Contratação", "data_contratacao") \
    .withColumnRenamed("Data Rescisão", "data_rescisao") \
    .withColumnRenamed("MotivRecisao", "motivo_rescisao") \
    .withColumnRenamed("SituaçãoEmprego", "situacao_emprego") \
    .withColumnRenamed("Departamento", "departamento") \
    .withColumnRenamed("Nome Gestor", "nome_gestor") \
    .withColumnRenamed("IDGestor", "id_gestor") \
    .withColumnRenamed("Fonte de recrutamento", "fonte_recrutamento") \
    .withColumnRenamed("Avaliacao Desempenho", "avaliacao_desempenho") \
    .withColumnRenamed("Pesquisa Engajamento", "pesquisa_engajamento") \
    .withColumnRenamed("Satisfação Funcionario", "satisfacao_funcionario") \
    .withColumnRenamed("Contagem de Projetos Especiais", "contagem_projetos_especiais") \
    .withColumnRenamed("DataUltimoFeedback", "data_ultimo_feedback") \
    .withColumnRenamed("UltimoAtrasoUltimos30dias", "dias_atrasado_ultimo_30") \
    .withColumnRenamed("Ausências", "ausencias")

# Conversão de tipos
df_spark = df_spark \
    .withColumn("id_funcionario", col("id_funcionario").cast(IntegerType())) \
    .withColumn(
        "salario",
        regexp_replace(
            regexp_replace(
                regexp_replace(col("salario"), r'[$\s]', ''),  # remove $ e espaços
                r'\.', ''  # remove ponto dos milhares
            ),
            ',', '.'  # substitui vírgula por ponto decimal
        ).cast(FloatType())
    ) \
    .withColumn("data_contratacao", to_date(col("data_contratacao"), "M/d/yyyy")) \
    .withColumn("data_rescisao", to_date(col("data_rescisao"), "M/d/yyyy")) \
    .withColumn("data_ultimo_feedback", to_date(col("data_ultimo_feedback"), "M/d/yyyy"))

# Grava os dados na tabela Delta da camada BRONZE
df_spark.write \
    .format("delta") \
    .mode("append") \
    .saveAsTable("bronze.rhg_dataset")

print("Dados carregados com sucesso na tabela bronze.rhg_dataset!")

Dimensões do dataset: (311, 36)
root
 |-- Nome do funcionário: string (nullable = true)
 |-- IDFuncionario: long (nullable = true)
 |-- IDCasado: long (nullable = true)
 |-- IDStatusConjulgal: long (nullable = true)
 |-- IDGenero: long (nullable = true)
 |-- IDStatusFuncionario: long (nullable = true)
 |-- IDepartamento: long (nullable = true)
 |-- IDPontuacaoDesempenho: long (nullable = true)
 |-- IDRecrutamentoDiversidade: long (nullable = true)
 |-- Salario: string (nullable = true)
 |-- IDesligado: long (nullable = true)
 |-- IDCargo: long (nullable = true)
 |-- Cargo: string (nullable = true)
 |-- Estado: string (nullable = true)
 |-- CEP: long (nullable = true)
 |-- ANIV: string (nullable = true)
 |-- Sexo: string (nullable = true)
 |-- EstadoCivil: string (nullable = true)
 |-- Cidadania: string (nullable = true)
 |-- Latino: string (nullable = true)
 |-- Raca: string (nullable = true)
 |-- Data Contratação: string (nullable = true)
 |-- Data Rescisão: string (nullable = true)
 

###Teste de carga da Tabela
Teste de carga da tabela, para garantir o sucesso da operação.

In [0]:
%sql
SELECT * 
FROM bronze.rhg_dataset
LIMIT 10;

nome_funcionario,id_funcionario,id_casado,id_status_conjulgal,id_genero,id_status_funcionario,id_departamento,id_pontuacao_desempenho,id_recrutamento_diversidade,salario,id_desligado,id_cargo,cargo,estado,cep,aniversario,sexo,estado_civil,cidadania,latino,raca,data_contratacao,data_rescisao,motivo_rescisao,situacao_emprego,departamento,nome_gestor,id_gestor,fonte_recrutamento,avaliacao_desempenho,pesquisa_engajamento,satisfacao_funcionario,contagem_projetos_especiais,data_ultimo_feedback,dias_atrasado_ultimo_30,ausencias
"Adinolfi, Wilson K",10026,0,0,1,1,5,4,0,62506.0,0,19,Técnico de Produção I,MA,1960,07/10/83,H,Solteiro,Cidadão americano,Não,Branco,2011-07-05,null,N/A - Ainda empregado,Ativo,Produção,Michael Albert,22.0,LinkedIn,Excede,460,Muito Satisfeito,0,2019-01-17,0,1
"Ait Sidi, Karthikeyan",10084,1,1,1,5,3,3,0,104437.0,1,27,Sr. DBA,MA,2148,05/05/75,H,Casado,Cidadão americano,Não,Branco,2015-03-30,2016-06-16,Mudança de carreira,Rescindido voluntariamente,T.I,Simon Roup,4.0,Indeed,Atende totalmente,496,Neutro,6,2016-02-24,0,17
"Akinkuolie, Sarah",10196,1,1,0,5,5,3,0,64955.0,1,20,Técnico de Produção II,MA,1810,09/19/88,M,Casado,Cidadão americano,Não,Branco,2011-07-05,2012-09-24,Horas,Rescindido voluntariamente,Produção,Kissy Sullivan,20.0,LinkedIn,Atende totalmente,302,Neutro,0,2012-05-15,0,3
"Alagbe,Trina",10088,1,1,0,1,5,3,0,64991.0,0,19,Técnico de Produção I,MA,1886,09/27/88,M,Casado,Cidadão americano,Não,Branco,2008-01-07,null,N/A - Ainda empregado,Ativo,Produção,Elijiah Gray,16.0,Indeed,Atende totalmente,484,Muito Satisfeito,0,2019-01-03,0,15
"Anderson, Carol",10069,0,2,0,5,5,3,0,50825.0,1,19,Técnico de Produção I,MA,2169,09/08/89,M,Divorciado,Cidadão americano,Não,Branco,2011-07-11,2016-09-06,Retorno à escola,Rescindido voluntariamente,Produção,Webster Butler,39.0,Pesquisa Google,Atende totalmente,500,Satisfeito,0,2016-02-01,0,2
"Anderson, Linda",10002,0,0,0,1,5,4,0,57568.0,0,19,Técnico de Produção I,MA,1844,05/22/77,M,Solteiro,Cidadão americano,Não,Branco,2012-01-09,null,N/A - Ainda empregado,Ativo,Produção,Amy Dunn,11.0,LinkedIn,Excede,500,Muito Satisfeito,0,2019-01-07,0,15
"Andreola, Colby",10194,0,0,0,1,4,3,0,95660.0,0,24,Engenheiro de software,MA,2110,05/24/79,M,Solteiro,Cidadão americano,Não,Branco,2014-11-10,null,N/A - Ainda empregado,Ativo,Engenharia de Software,Alex Sweetwater,10.0,LinkedIn,Atende totalmente,304,Neutro,4,2019-01-02,0,19
"Athwal, Sam",10062,0,4,1,1,5,3,0,59365.0,0,19,Técnico de Produção I,MA,2199,02/18/83,H,Viúva,Cidadão americano,Não,Branco,2013-09-30,null,N/A - Ainda empregado,Ativo,Produção,Ketsia Liebig,19.0,Indicação de funcionários,Atende totalmente,500,Satisfeito,0,2019-02-25,0,19
"Bachiochi, Linda",10114,0,0,0,3,5,3,1,47837.0,0,19,Técnico de Produção I,MA,1902,02/11/70,M,Solteiro,Cidadão americano,Não,Negro ou afro-americano,2009-07-06,null,N/A - Ainda empregado,Ativo,Produção,Brannon Miller,12.0,Feira de Empregos de Diversidade,Atende totalmente,446,Neutro,0,2019-01-25,0,4
"Bacong, Alejandro",10250,0,2,1,1,3,3,0,50178.0,0,14,Suporte de TI,MA,1886,01/07/88,H,Divorciado,Cidadão americano,Não,Branco,2015-01-05,null,N/A - Ainda empregado,Ativo,T.I,Peter Monroe,7.0,Indeed,Atende totalmente,500,Muito Satisfeito,6,2019-02-18,0,16
